<a href="https://colab.research.google.com/github/Eddy-Emmanuel/Movie-NER-Transformers/blob/main/Movie_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install datasets
!pip -q install evaluate
!pip -q install seqeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12=

In [2]:
import evaluate
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (AutoTokenizer,
                          AutoModelForTokenClassification,
                          TrainingArguments,
                          Trainer,
                          DataCollatorForTokenClassification)
from sklearn.model_selection import train_test_split

In [3]:
movie_dataset = load_dataset("ttxy/ner_movie")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/147 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/431 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/441k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7816 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1953 [00:00<?, ? examples/s]

In [4]:
movie_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7816
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1953
    })
})

In [5]:
movie_dataset["train"].to_pandas().head()

,text,label
0,steve mcqueen provided a thrilling motorcycle ...,"B-Actor,I-Actor,O,O,B-Plot,I-Plot,I-Plot,I-Plo..."
1,liza minnelli and joel gray won oscars for the...,"B-Actor,I-Actor,O,B-Actor,I-Actor,B-Award,I-Aw..."
2,what is that tom hanks and julia roberts movie...,"O,O,O,B-Actor,I-Actor,O,B-Actor,I-Actor,O,O,B-..."
3,what is the movie making fun of macgyver by re...,"O,O,O,O,B-Plot,I-Plot,I-Plot,I-Plot,I-Plot,I-P..."
4,i am thinking of an animated film based on a c...,"O,O,O,O,O,B-Genre,O,B-Origin,I-Origin,I-Origin..."


In [6]:
df = movie_dataset["train"].to_pandas()

In [7]:
all_labels = list(set("".join(df["label"]).split(",")))

print(all_labels)

['I-Actor', 'B-OpinionB-Plot', 'B-Director', 'B-Relationship', 'B-PlotO', 'B-RelationshipB-Character_Name', 'I-DirectorB-Origin', 'I-OriginB-Opinion', 'OB-Year', 'I-RelationshipB-Plot', 'B-Character_NameB-Plot', 'I-GenreB-Origin', 'I-DirectorB-Actor', 'B-Year', 'I-Relationship', 'B-Plot', 'I-QuoteO', 'OB-Quote', 'I-GenreB-Genre', 'B-GenreB-Actor', 'B-Character_NameO', 'I-SoundtrackB-Actor', 'I-Award', 'I-ActorB-Actor', 'B-Quote', 'I-Soundtrack', 'I-GenreO', 'I-YearO', 'I-DirectorB-Year', 'B-ActorO', 'I-DirectorB-Genre', 'B-AwardO', 'I-RelationshipB-Director', 'B-GenreB-Plot', 'B-OpinionB-Actor', 'I-OriginB-Character_Name', 'B-QuoteO', 'I-DirectorB-Plot', 'B-Origin', 'B-OriginB-Plot', 'I-PlotB-Plot', 'I-OriginB-Relationship', 'OB-Actor', 'I-OpinionB-Actor', 'I-AwardO', 'I-Character_NameB-Character_Name', 'B-OriginO', 'B-OpinionB-Director', 'B-YearB-Actor', 'I-OriginB-Genre', 'OO', 'B-DirectorB-Plot', 'B-YearB-Plot', 'OB-Character_Name', 'I-DirectorB-Director', 'B-Actor', 'I-Genre', 'I-P

In [8]:
idx2label = {i:j for i,j in enumerate(all_labels)}
label2idx = {j:i for i,j in enumerate(all_labels)}

In [9]:
movie_dataset = movie_dataset.map(lambda x: {"label_ids":[label2idx[i] for i in x["label"].split(",")]})
movie_dataset

Map:   0%|          | 0/7816 [00:00<?, ? examples/s]

Map:   0%|          | 0/1953 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_ids'],
        num_rows: 7816
    })
    test: Dataset({
        features: ['text', 'label', 'label_ids'],
        num_rows: 1953
    })
})

In [10]:
movie_dataset["train"].to_pandas().head()

,text,label,label_ids
0,steve mcqueen provided a thrilling motorcycle ...,"B-Actor,I-Actor,O,O,B-Plot,I-Plot,I-Plot,I-Plo...","[55, 0, 88, 88, 15, 105, 105, 105, 105, 116, 6..."
1,liza minnelli and joel gray won oscars for the...,"B-Actor,I-Actor,O,B-Actor,I-Actor,B-Award,I-Aw...","[55, 0, 88, 55, 0, 70, 22, 88, 88, 88, 88, 88,..."
2,what is that tom hanks and julia roberts movie...,"O,O,O,B-Actor,I-Actor,O,B-Actor,I-Actor,O,O,B-...","[88, 88, 88, 55, 0, 88, 55, 0, 88, 88, 15, 105..."
3,what is the movie making fun of macgyver by re...,"O,O,O,O,B-Plot,I-Plot,I-Plot,I-Plot,I-Plot,I-P...","[88, 88, 88, 88, 15, 105, 105, 105, 105, 105, ..."
4,i am thinking of an animated film based on a c...,"O,O,O,O,O,B-Genre,O,B-Origin,I-Origin,I-Origin...","[88, 88, 88, 88, 88, 123, 88, 38, 87, 87, 87, ..."


In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
tokenizer(movie_dataset["train"][0]["text"])

{'input_ids': [101, 3889, 29265, 3024, 1037, 26162, 9055, 5252, 1999, 2023, 4602, 1997, 2035, 1059, 2860, 1016, 3827, 4019, 5691, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
def AlignLabelWithToken(batch):
    tokenized_inputs = tokenizer(batch["text"], truncation=True, is_split_into_words=False)

    aligned_labels = []
    for idx, label_ids in enumerate(batch["label_ids"]):
        labels = []
        previous_wrd = None
        for wrd_id in tokenized_inputs.word_ids(batch_index=idx):
            if wrd_id is None:
                labels.append(-100)
            elif wrd_id != previous_wrd:
                labels.append(label_ids[wrd_id])
            else:
                labels.append(label_ids[wrd_id])

            previous_wrd = wrd_id

        aligned_labels.append(labels)

    tokenized_inputs["labels"] = aligned_labels

    return tokenized_inputs

In [14]:
movie_dataset = movie_dataset.map(AlignLabelWithToken, batched=True)

movie_dataset

Map:   0%|          | 0/7816 [00:00<?, ? examples/s]

Map:   0%|          | 0/1953 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_ids', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 7816
    })
    test: Dataset({
        features: ['text', 'label', 'label_ids', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1953
    })
})

In [15]:
movie_dataset = movie_dataset.remove_columns(['text', 'label', 'label_ids', "token_type_ids"])
movie_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7816
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1953
    })
})

In [16]:
movie_dataset_df = movie_dataset["train"].to_pandas()

movie_dataset_df.head()

,input_ids,attention_mask,labels
0,"[101, 3889, 29265, 3024, 1037, 26162, 9055, 52...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 55, 0, 88, 88, 15, 105, 105, 105, 105, ..."
1,"[101, 20503, 8117, 9091, 2072, 1998, 8963, 389...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 55, 0, 0, 0, 88, 55, 0, 70, 22, 22, 88,..."
2,"[101, 2054, 2003, 2008, 3419, 9180, 2015, 1998...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 88, 88, 88, 55, 0, 0, 88, 55, 0, 88, 88..."
3,"[101, 2054, 2003, 1996, 3185, 2437, 4569, 1997...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 88, 88, 88, 88, 15, 105, 105, 105, 105,..."
4,"[101, 1045, 2572, 3241, 1997, 2019, 6579, 2143...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 88, 88, 88, 88, 88, 123, 88, 38, 87, 87..."


In [17]:
Xtrain, Xtest = train_test_split(movie_dataset_df, test_size=0.2, random_state=2025)

print(Xtrain.shape, Xtest.shape)

(6252, 3) (1564, 3)


In [18]:
Dataset.from_pandas(Xtrain.reset_index(drop=True))

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 6252
})

In [19]:
input_data = DatasetDict(
    {"train":Dataset.from_pandas(Xtrain.reset_index(drop=True)),
     "val":Dataset.from_pandas(Xtest.reset_index(drop=True)),
     "test":movie_dataset["test"]}
)

input_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 6252
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1564
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1953
    })
})

In [20]:
model =  AutoModelForTokenClassification.from_pretrained("bert-base-uncased",
                                                         num_labels=len(all_labels),
                                                         id2label=idx2label,
                                                         label2id=label2idx)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
all_metrics = evaluate.load("seqeval")

In [22]:
def compute_metrics(pred):
    labels, predictions = pred.label_ids, np.argmax(pred.predictions, axis=2)

    y_true = [
        [idx2label[l] for l, p in zip(label, pred) if l != -100] for label, pred in zip(labels, predictions)
    ]
    y_pred = [
        [idx2label[p] for l, p in zip(label, pred) if l != -100] for label, pred in zip(labels, predictions)
    ]

    results = all_metrics.compute(predictions=y_pred, references=y_true)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }

In [25]:
training_arg = TrainingArguments(output_dir="movie_ner",
                                 per_device_train_batch_size=16,
                                 per_device_eval_batch_size=16,
                                 learning_rate=2e-5,
                                 weight_decay=0.01,
                                 lr_scheduler_type="reduce_lr_on_plateau",
                                 eval_strategy="epoch",
                                 logging_strategy="epoch",
                                 save_strategy="epoch",
                                 load_best_model_at_end=True,
                                 num_train_epochs=5)


trainer = Trainer(model=model,
                  args=training_arg,
                  processing_class=tokenizer,
                  compute_metrics=compute_metrics,
                  eval_dataset=input_data["val"],
                  train_dataset=input_data["train"],
                  data_collator=DataCollatorForTokenClassification(tokenizer))

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.095300,0.526861,0.674170,0.715489,0.694215,0.873054
2,0.072400,0.595881,0.680532,0.720775,0.700076,0.875000
3,0.057600,0.593651,0.686844,0.717642,0.701906,0.875944
4,0.046700,0.645114,0.683502,0.715489,0.699129,0.872847
5,0.039700,0.662478,0.651765,0.705111,0.677389,0.870842


TrainOutput(global_step=1955, training_loss=0.06233741730985129, metrics={'train_runtime': 371.4876, 'train_samples_per_second': 84.148, 'train_steps_per_second': 5.263, 'total_flos': 598136056817904.0, 'train_loss': 0.06233741730985129, 'epoch': 5.0})

In [27]:
import pandas as pd

In [28]:
pd.DataFrame([trainer.evaluate(input_data["test"])])

,eval_loss,eval_precision,eval_recall,eval_f1,eval_accuracy,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,0.504294,0.668333,0.718262,0.692398,0.876514,5.1339,380.413,23.958,5.0


In [29]:
trainer.save_model("movie_ner_model")

#### Load trained model using pipeline

In [30]:
from transformers import pipeline

In [31]:
pipe = pipeline("ner",
                model="/content/movie_ner_model",
                tokenizer="/content/movie_ner_model")

pipe

Device set to use cuda:0


In [32]:
sample_text = """
Here's a list of some popular and well-known movie actors: Mark Wahlberg,
Johnny Depp, Chris Evans, Daniel Evans, Fran Drescher, Keith David, Peter Coyote,
Tom Cruise, Adam Sandler, Amy Adams, Ashley Benson, Ben Affleck, Bill Duke, Blake Jenner, Cate Blanchett.
Mark Wahlberg, Johnny Depp, Chris Evans, Daniel Evans, Fran Drescher, Keith David, Peter Coyote, Tom Cruise,
Adam Sandler, Amy Adams, Ashley Benson, Ben Affleck, Bill Duke, Blake Jenner, Cate Blanchett, Morgan Freeman,
Samuel L. Jackson, Denzel Washington, Keanu Reeves, Harrison Ford, Bruce Willis, Anthony Hopkins, Danny DeVito, Brad Pitt,
Leonardo DiCaprio, Michael Caine, Matt Damon, Robert De Niro, Tom Hanks, and Christian Bale.
"""

results = pipe(sample_text)

results

[{'entity': 'B-Opinion',
  'score': np.float32(0.5414926),
  'index': 8,
  'word': 'popular',
  'start': 23,
  'end': 30},
 {'entity': 'B-Actor',
  'score': np.float32(0.99897885),
  'index': 16,
  'word': 'mark',
  'start': 60,
  'end': 64},
 {'entity': 'I-Actor',
  'score': np.float32(0.99930453),
  'index': 17,
  'word': 'wah',
  'start': 65,
  'end': 68},
 {'entity': 'I-Actor',
  'score': np.float32(0.9993205),
  'index': 18,
  'word': '##lberg',
  'start': 68,
  'end': 73},
 {'entity': 'B-Actor',
  'score': np.float32(0.99811685),
  'index': 20,
  'word': 'johnny',
  'start': 75,
  'end': 81},
 {'entity': 'I-Actor',
  'score': np.float32(0.99933356),
  'index': 21,
  'word': 'de',
  'start': 82,
  'end': 84},
 {'entity': 'I-Actor',
  'score': np.float32(0.9993332),
  'index': 22,
  'word': '##pp',
  'start': 84,
  'end': 86},
 {'entity': 'B-Actor',
  'score': np.float32(0.99773955),
  'index': 24,
  'word': 'chris',
  'start': 88,
  'end': 93},
 {'entity': 'I-Actor',
  'score': np

#### Load model using Transformers

In [37]:
loaded_tokenizer = AutoTokenizer.from_pretrained("/content/movie_ner_model")
loaded_model = AutoModelForTokenClassification.from_pretrained("/content/movie_ner_model")

In [44]:
import torch

In [52]:
tokenized_sample = loaded_tokenizer(sample_text, return_tensors="pt")

# Run Prediction
with torch.no_grad():
    outputs = loaded_model(**tokenized_sample)

predictions = torch.argmax(outputs.logits, dim=2).squeeze().tolist()

In [53]:
tokenized_sample

{'input_ids': tensor([[  101,  2182,  1005,  1055,  1037,  2862,  1997,  2070,  2759,  1998,
          2092,  1011,  2124,  3185,  5889,  1024,  2928, 22894, 22927,  1010,
          5206,  2139,  9397,  1010,  3782,  6473,  1010,  3817,  6473,  1010,
         23151,  2852,  2229,  7474,  1010,  6766,  2585,  1010,  2848, 20457,
          1010,  3419,  8592,  1010,  4205,  5472,  3917,  1010,  6864,  5922,
          1010,  9321, 11999,  1010,  3841, 21358, 21031,  3600,  1010,  3021,
          3804,  1010,  6511, 15419,  3678,  1010,  4937,  2063, 18158,  4779,
          1012,  2928, 22894, 22927,  1010,  5206,  2139,  9397,  1010,  3782,
          6473,  1010,  3817,  6473,  1010, 23151,  2852,  2229,  7474,  1010,
          6766,  2585,  1010,  2848, 20457,  1010,  3419,  8592,  1010,  4205,
          5472,  3917,  1010,  6864,  5922,  1010,  9321, 11999,  1010,  3841,
         21358, 21031,  3600,  1010,  3021,  3804,  1010,  6511, 15419,  3678,
          1010,  4937,  2063, 18158,  

In [49]:
outputs

TokenClassifierOutput(loss=None, logits=tensor([[[ 1.0819, -3.8920,  0.4461,  ..., -4.4443, -4.0300, -3.9987],
         [ 0.5655, -3.9448, -0.0743,  ..., -4.1486, -3.9694, -3.6569],
         [ 0.6771, -4.3415, -0.2944,  ..., -4.2430, -4.1480, -3.8147],
         ...,
         [11.2632, -1.9087, -0.3847,  ..., -1.9440, -1.7484, -1.7106],
         [ 1.3445, -4.3727, -0.0612,  ..., -4.5731, -4.1750, -4.0140],
         [ 0.8233, -1.0304,  0.0377,  ..., -1.1034, -1.5080, -0.7323]]]), hidden_states=None, attentions=None)

In [56]:
print(predictions)

[88, 88, 88, 88, 88, 88, 88, 88, 116, 88, 88, 88, 88, 88, 88, 88, 55, 0, 0, 88, 55, 0, 0, 88, 55, 0, 88, 55, 0, 88, 55, 0, 0, 0, 88, 55, 0, 88, 55, 0, 88, 55, 0, 88, 55, 0, 0, 88, 55, 0, 88, 55, 0, 88, 55, 0, 0, 0, 88, 55, 0, 88, 55, 0, 0, 88, 55, 55, 0, 0, 88, 55, 0, 0, 88, 55, 0, 0, 88, 55, 0, 88, 55, 0, 88, 55, 0, 0, 0, 88, 0, 0, 88, 55, 0, 88, 55, 0, 88, 55, 0, 0, 88, 55, 0, 88, 55, 0, 88, 55, 0, 0, 0, 88, 55, 0, 88, 55, 0, 0, 88, 55, 55, 0, 0, 88, 55, 0, 88, 55, 0, 3, 0, 88, 55, 55, 0, 88, 55, 55, 0, 88, 55, 0, 88, 55, 0, 88, 55, 0, 88, 55, 0, 0, 88, 55, 0, 88, 55, 0, 0, 0, 88, 55, 0, 88, 55, 0, 88, 55, 0, 0, 0, 88, 55, 0, 0, 3, 88, 55, 0, 0, 88, 3]


In [60]:
tokenized_sample["input_ids"].numpy()[0]

array([  101,  2182,  1005,  1055,  1037,  2862,  1997,  2070,  2759,
        1998,  2092,  1011,  2124,  3185,  5889,  1024,  2928, 22894,
       22927,  1010,  5206,  2139,  9397,  1010,  3782,  6473,  1010,
        3817,  6473,  1010, 23151,  2852,  2229,  7474,  1010,  6766,
        2585,  1010,  2848, 20457,  1010,  3419,  8592,  1010,  4205,
        5472,  3917,  1010,  6864,  5922,  1010,  9321, 11999,  1010,
        3841, 21358, 21031,  3600,  1010,  3021,  3804,  1010,  6511,
       15419,  3678,  1010,  4937,  2063, 18158,  4779,  1012,  2928,
       22894, 22927,  1010,  5206,  2139,  9397,  1010,  3782,  6473,
        1010,  3817,  6473,  1010, 23151,  2852,  2229,  7474,  1010,
        6766,  2585,  1010,  2848, 20457,  1010,  3419,  8592,  1010,
        4205,  5472,  3917,  1010,  6864,  5922,  1010,  9321, 11999,
        1010,  3841, 21358, 21031,  3600,  1010,  3021,  3804,  1010,
        6511, 15419,  3678,  1010,  4937,  2063, 18158,  4779,  1010,
        5253, 11462,

In [62]:
# Decode Tokens and Labels
tokens = loaded_tokenizer.convert_ids_to_tokens(tokenized_sample["input_ids"].numpy()[0])
predicted_labels = [loaded_model.config.id2label[pred] for pred in predictions]

# Print Results
for token, label in zip(tokens, predicted_labels):
    print(f"{token}: {label}")

[CLS]: O
here: O
': O
s: O
a: O
list: O
of: O
some: O
popular: B-Opinion
and: O
well: O
-: O
known: O
movie: O
actors: O
:: O
mark: B-Actor
wah: I-Actor
##lberg: I-Actor
,: O
johnny: B-Actor
de: I-Actor
##pp: I-Actor
,: O
chris: B-Actor
evans: I-Actor
,: O
daniel: B-Actor
evans: I-Actor
,: O
fran: B-Actor
dr: I-Actor
##es: I-Actor
##cher: I-Actor
,: O
keith: B-Actor
david: I-Actor
,: O
peter: B-Actor
coyote: I-Actor
,: O
tom: B-Actor
cruise: I-Actor
,: O
adam: B-Actor
sand: I-Actor
##ler: I-Actor
,: O
amy: B-Actor
adams: I-Actor
,: O
ashley: B-Actor
benson: I-Actor
,: O
ben: B-Actor
af: I-Actor
##fle: I-Actor
##ck: I-Actor
,: O
bill: B-Actor
duke: I-Actor
,: O
blake: B-Actor
jen: I-Actor
##ner: I-Actor
,: O
cat: B-Actor
##e: B-Actor
blanche: I-Actor
##tt: I-Actor
.: O
mark: B-Actor
wah: I-Actor
##lberg: I-Actor
,: O
johnny: B-Actor
de: I-Actor
##pp: I-Actor
,: O
chris: B-Actor
evans: I-Actor
,: O
daniel: B-Actor
evans: I-Actor
,: O
fran: B-Actor
dr: I-Actor
##es: I-Actor
##cher: I-Acto

In [63]:
loaded_model.config

BertConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "I-Actor",
    "1": "B-OpinionB-Plot",
    "2": "B-Director",
    "3": "B-Relationship",
    "4": "B-PlotO",
    "5": "B-RelationshipB-Character_Name",
    "6": "I-DirectorB-Origin",
    "7": "I-OriginB-Opinion",
    "8": "OB-Year",
    "9": "I-RelationshipB-Plot",
    "10": "B-Character_NameB-Plot",
    "11": "I-GenreB-Origin",
    "12": "I-DirectorB-Actor",
    "13": "B-Year",
    "14": "I-Relationship",
    "15": "B-Plot",
    "16": "I-QuoteO",
    "17": "OB-Quote",
    "18": "I-GenreB-Genre",
    "19": "B-GenreB-Actor",
    "20": "B-Character_NameO",
    "21": "I-SoundtrackB-Actor",
    "22": "I-Award",
    "23": "I-ActorB-Actor",
    "24": "B-Quote",
    "25": "I-Soundt